<a href="https://colab.research.google.com/github/playx1345/git-hub-test/blob/main/Supabase_Storage_migration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Step 1 - Install required packages for the migration process

In [12]:
!apt-get update && apt-get install -y libmagic1
!pip install python-magic supabase

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:5 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:7 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 3,917 B in 1s (2,976 B/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (

### Step 2 - Import required packages

In [13]:
import re
import os
import magic
import shutil
import zipfile
from typing import Optional
from pathlib import Path
from google.colab import files
from supabase import create_client

print("✓ Imports are set!")

✓ Imports are set!


### Step 3 - Setup Supabase database credentials

In [ ]:
NEW_PROJECT_URL = "https://[PROJECT_REF].supabase.co" # @param {type:"string"}
NEW_SERVICE_KEY = "SERVICE_ROLE_KEY" # @param {type:"string"}

print("✓ New Project Credentials are set!")

### Step 4 - Storage migration script

In [11]:
# Initialize Supabase client
try:
    new_supabase_client = create_client(NEW_PROJECT_URL, NEW_SERVICE_KEY)
except Exception as e:
    raise Exception(f"Failed to initialize Supabase client: {str(e)}")

# Upload and extract files
local_root_path = upload_to_colab()

# List existing buckets
try:
    buckets = new_supabase_client.storage.list_buckets()
except Exception as e:
    raise Exception(f"Failed to list buckets: {str(e)}")

if not local_root_path.exists():
    raise FileNotFoundError(f"Local root path does not exist: {local_root_path}")

print("\nFound the following buckets:")
for bucket in buckets:
    print(f"- {bucket.name}")

for bucket in buckets:
    bucket_name = bucket.name
    local_bucket_path = local_root_path /bucket_name
    print(f"\nLocal path: '{local_bucket_path}'")
    if not local_bucket_path.exists():
        print(f"\nLocal path for bucket '{bucket_name}' does not exist. Skipping.")
        continue

    print(f"\nProcessing bucket '{bucket_name}'...")

    # Create bucket if it doesn't exist
    try:
        new_supabase_client.storage.create_bucket(
            bucket_name,
            options={"public": bucket.public}
        )
        print(f"Bucket '{bucket_name}' created successfully.")
    except Exception as e:
        if "already exists" not in str(e).lower():
            print(f"Error creating bucket '{bucket_name}': {str(e)}")
            continue

    # Upload files
    upload_local_files_to_bucket(
        new_supabase_client,
        bucket_name,
        str(local_bucket_path),
        skip_existing=True
    )

def upload_to_colab():
    """Upload and extract ZIP file containing storage buckets"""
    print("Please select your storage ZIP file containing all bucket folders...")
    uploaded = files.upload()

    if not uploaded:
        raise ValueError("No file was uploaded")

    # Get the first uploaded file
    zip_filename = next(iter(uploaded))

    # Create a temporary directory for extraction
    temp_dir = Path("/content/storage_temp")
    if temp_dir.exists():
        shutil.rmtree(temp_dir)
    temp_dir.mkdir()

    # Extract the ZIP file
    with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
        zip_ref.extractall(temp_dir)

    print(f"\nFiles extracted to {temp_dir}")

    root_candidates = [d for d in temp_dir.iterdir() if d.is_dir()]
    root_dir = root_candidates[0]

    print(f"\nNew root dir is {root_dir}")

    return root_dir

def upload_local_files_to_bucket(
    client,
    bucket_name: str,
    local_bucket_path: str,
    skip_existing: bool = False
) -> None:
    """Upload files from a local directory to a specific Supabase storage bucket."""
    local_bucket_path = Path(local_bucket_path)

    if not local_bucket_path.exists():
        raise FileNotFoundError(f"Local bucket path does not exist: {local_bucket_path}")

    total_files = sum(1 for _ in local_bucket_path.rglob('*') if _.is_file())
    processed_files = 0

    for file_path in local_bucket_path.rglob('*'):
        if not file_path.is_file():
            continue

        try:
            relative_path = str(file_path.relative_to(local_bucket_path))
            remote_path = relative_path.replace(os.sep, '/')

            processed_files += 1
            print(f"\rProgress: {processed_files}/{total_files} files", end="")

            if skip_existing:
                try:
                    client.storage.from_(bucket_name).download(remote_path)
                    print(f"\nSkipping existing file: {remote_path}")
                    continue
                except Exception:
                    pass

            mime_type = magic.from_file(str(file_path), mime=True)

            with open(file_path, 'rb') as file_object:
                client.storage.from_(bucket_name).upload(
                    remote_path,
                    file_object,
                    file_options={"content-type": mime_type, "x-upsert": "true"}
                )
        except Exception as e:
            print(f"\nError uploading {file_path}: {str(e)}")

    print(f"\nCompleted uploading files to bucket '{bucket_name}'")

Please select your storage ZIP file containing all bucket folders...


TypeError: 'NoneType' object is not subscriptable